In [1]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import ResNet152, preprocess_input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.activations import relu, softmax, sigmoid, swish
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing import image

2022-11-30 09:54:57.774428: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
import warnings

# Ignore FutureWarning from numpy
warnings.simplefilter(action='ignore', category=FutureWarning)

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";

# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="0";

# Allow growth of GPU memory, otherwise it will always look like all the memory is being used
physical_devices = tf.config.experimental.list_physical_devices('GPU')

tf.config.experimental.set_memory_growth(physical_devices[0], True)

2022-11-30 09:55:07.046952: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-11-30 09:55:07.049829: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-11-30 09:55:07.062774: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2022-11-30 09:55:07.062837: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: lnx00273.ad.liu.se
2022-11-30 09:55:07.062854: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: lnx00273.ad.liu.se
2022-11-30 09:55:07.062994: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 525.60.11
2022-11-30 09:55:07.063071: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 515.86.

IndexError: list index out of range

In [3]:
datagenerator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    horizontal_flip=True,
    rotation_range=20,
    vertical_flip=True,
    width_shift_range=0.1,
    #featurewise_center=True,
    samplewise_center=True,
    samplewise_std_normalization=True,
    brightness_range=[0.90,1.25],
    fill_mode='nearest'
)

In [4]:
# load and iterate training dataset
train_data = datagenerator.flow_from_directory('../train/', 
    class_mode='binary',
    target_size=(224, 224), 
    batch_size=32, 
    shuffle=True)

# load and iterate validation dataset
val_data = datagenerator.flow_from_directory('../valid/', 
    class_mode='binary',
    target_size=(224, 224),
    batch_size=1, 
    shuffle=False)

# load and iterate test dataset
test_data = datagenerator.flow_from_directory('../test/', 
    class_mode='binary',
    target_size=(224, 224),
    batch_size=1, 
    shuffle=False)

Found 2600 images belonging to 2 classes.
Found 870 images belonging to 2 classes.
Found 864 images belonging to 2 classes.


In [5]:
# #sns.set_style('white')
# generated_image, label = val_data.__getitem__(0)
# plt.imshow(generated_image[0])

# plt.colorbar()
# plt.title('Raw Chest X Ray Image')

# print(f"The dimensions of the image are {generated_image.shape[1]} pixels width and {generated_image.shape[2]} pixels height, one single color channel.")
# print(f"The maximum pixel value is {generated_image.max():.4f} and the minimum is {generated_image.min():.4f}")
# print(f"The mean value of the pixels is {generated_image.mean():.4f} and the standard deviation is {generated_image.std():.4f}")

In [6]:
# generated_image.shape

In [7]:
pre_trained_model_resnet152 = ResNet152(input_shape=(224,224,3),
                                include_top=False,
                                weights="imagenet")

2022-11-29 11:16:11.553474: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-29 11:16:11.553772: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-11-29 11:16:11.553986: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-29 11:16:11.554145: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:02:00.0 name: NVIDIA GeForce RTX 3060 Ti computeCapability: 8.6
coreClock: 1.755GHz coreCount: 38 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.

In [8]:
# Some weights in later layers are unfreezed
for layer in pre_trained_model_resnet152.layers:
    layer.trainable=False

tf.random.set_seed(15)

model = tf.keras.models.Sequential([
    pre_trained_model_resnet152,
    GlobalAveragePooling2D(),    
    Dense(512,activation="relu"),
    Dropout(0.4),
    Dense(256,activation="relu"),
    Dropout(0.4),
    Dense(128, activation='swish'),  
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=RMSprop(learning_rate=1e-4),
              loss="binary_crossentropy",
              metrics=['accuracy'])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet152 (Functional)       (None, 7, 7, 2048)        58370944  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               1049088   
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               3

In [10]:
history = model.fit(
    train_data,
    epochs=5,
    validation_data=val_data  
)

2022-11-29 11:17:08.266775: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-11-29 11:17:08.267169: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3491940000 Hz


Epoch 1/5


2022-11-29 11:17:15.201532: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-11-29 11:17:19.384881: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2022-11-29 11:25:02.874908: I tensorflow/stream_executor/cuda/cuda_driver.cc:789] failed to allocate 512.00M (536870912 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-11-29 11:25:02.890072: I tensorflow/stream_executor/cuda/cuda_driver.cc:789] failed to allocate 460.80M (483183872 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-11-29 11:25:03.004020: W tensorflow/stream_executor/gpu/asm_compiler.cc:63] Running ptxas --version returned 256
2022-11-29 11:25:03.105872: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location

82/82 [==============================] - ETA: 0s - loss: 239468.9575 - accuracy: 0.7837

2022-11-29 11:31:02.945611: W tensorflow/core/common_runtime/bfc_allocator.cc:314] Garbage collection: deallocate free memory regions (i.e., allocations) so that we can re-allocate a larger region to avoid OOM due to memory fragmentation. If you see this message frequently, you are running near the threshold of the available device memory and re-allocation may incur great performance overhead. You may try smaller batch sizes to observe the performance impact. Set TF_ENABLE_GPU_GARBAGE_COLLECTION=false if you'd like to disable this feature.
2022-11-29 11:31:02.960159: I tensorflow/stream_executor/cuda/cuda_driver.cc:789] failed to allocate 548.06M (574685184 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-11-29 11:31:14.715341: I tensorflow/stream_executor/cuda/cuda_driver.cc:789] failed to allocate 548.06M (574685184 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory


82/82 [==============================] - 995s 6s/step - loss: 237408.5874 - accuracy: 0.7838 - val_loss: 0.6617 - val_accuracy: 0.7989
Epoch 2/5


2022-11-29 11:33:47.224988: I tensorflow/stream_executor/cuda/cuda_driver.cc:789] failed to allocate 548.06M (574685184 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-11-29 11:33:47.226360: I tensorflow/stream_executor/cuda/cuda_driver.cc:789] failed to allocate 548.06M (574685184 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-11-29 11:33:47.227730: I tensorflow/stream_executor/cuda/cuda_driver.cc:789] failed to allocate 548.06M (574685184 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-11-29 11:33:57.227339: I tensorflow/stream_executor/cuda/cuda_driver.cc:789] failed to allocate 548.06M (574685184 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-11-29 11:33:57.228674: I tensorflow/stream_executor/cuda/cuda_driver.cc:789] failed to allocate 548.06M (574685184 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-11-29 11:33:57.228723: W tensorflow/core/common_runtime/bfc_allocator.cc:433] Allocator (GPU_0_

ResourceExhaustedError:  OOM when allocating tensor with shape[32,64,112,112] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node sequential/resnet152/conv1_bn/FusedBatchNormV3 (defined at tmp/ipykernel_29837/720030878.py:1) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_28093]

Function call stack:
train_function


In [ ]:
model.evaluate(test_data)

864/864 [==============================] - 159s 184ms/step - loss: 0.5134 - accuracy: 0.7998


[0.5134024024009705, 0.7997685074806213]